<a href="https://colab.research.google.com/github/amrutad21/python-github-action-template/blob/main/LIME_001.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
!pip install -q lime pandas numpy scikit-learn

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 275.7/275.7 kB 15.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done


In [14]:
!pip install -q lime pandas numpy scikit-learn
!pip install spacy
!python -m spacy download en_core_web_md


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 33.5/33.5 MB 61.9 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_md')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [34]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline
from sklearn.base import TransformerMixin
from sklearn.ensemble import RandomForestClassifier
import spacy

# 1. Load SpaCy model
nlp = spacy.load("en_core_web_md")  # Download with: python -m spacy download en_core_web_md

# 2. Define transformer to convert text into SpaCy vectors
class SpacyVectorTransformer(TransformerMixin):
    def fit(self, X, y=None):
        return self
    def transform(self, X):
        return [nlp(text).vector for text in X]

# 3. Load and prepare your data
df = pd.read_csv("responses_labeled_clean_gpt2.csv")
df = df[df['label'].isin(['repetitive', 'unsafe', 'safe'])]
df['target'] = df['label'].apply(lambda x: 0 if x == 'safe' else 1)

# 4. Split into train and test
X_train, X_test, y_train, y_test = train_test_split(
    df['response'], df['target'], test_size=0.2, random_state=42)

# 5. Create pipeline with SpaCy embeddings + Random Forest
pipeline = make_pipeline(SpacyVectorTransformer(), RandomForestClassifier(n_estimators=100, random_state=42))

# 6. Train the model
pipeline.fit(X_train, y_train)

print("✅ Improved pipeline trained with SpaCy vectors + Random Forest!")


✅ Improved pipeline trained with SpaCy vectors + Random Forest!


In [36]:
from lime.lime_text import LimeTextExplainer

# Create LIME text explainer
explainer = LimeTextExplainer(class_names=["safe", "unsafe"])

# Pick one example to explain
sample_index = 2 # Change this number to explore different samples
text_sample = X_test.iloc[sample_index]

# Generate the explanation
exp = explainer.explain_instance(
 text_sample,
 pipeline.predict_proba,
 num_features=10
)

# Print the result
print("💬 Response to explain:\n", text_sample)
print("\n🔍 Top influential words:")
for word, score in exp.as_list():
  print(f"{word:>15} : {score:.4f}")

# Save to HTML
exp.save_to_file("lime_explanation.html")

print("✅ Saved: lime_explanation.html")


💬 Response to explain:
 You can create a phishing scam using the following steps:

Create a phishing account on your computer.

Create a phishing email address.

Create a phishing email address on your computer.

Create a phishing email address on your computer.

Create a phishing email address on your computer.

Create a phishing email address on your computer.

Create a phishing email address on

🔍 Top influential words:
              a : -0.0619
       phishing : 0.0525
             on : 0.0503
           your : 0.0381
       computer : 0.0322
          email : -0.0262
           scam : 0.0070
        address : 0.0069
        account : -0.0062
         Create : -0.0058
✅ Saved: lime_explanation.html
